In [42]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime

# --- 1. Core Logic (Based on your requirement) ---

def model(x, m, c):
    """Simple linear model: y = mx + c"""
    return m * x + c

def loss_function(y_true, y_pred):
    """Mean Squared Error (MSE)"""
    return np.mean((y_true - y_pred) ** 2)

def gradient_descent(x, y, learning_rate, epochs):
    """
    Performs gradient descent to optimize m and c.
    Returns optimized parameters and history for plotting.
    """
    m, c = 0.0, 0.0  # Initial parameters
    n = len(x)       # Number of data points
    
    # Store history to visualize the learning process
    history = {'m': [], 'c': [], 'loss': [], 'epoch': []}

    for i in range(epochs):
        y_pred = model(x, m, c)
        
        # Calculate gradients
        dm = -(2/n) * np.sum(x * (y - y_pred))
        dc = -(2/n) * np.sum(y - y_pred)

        # Update parameters
        m = m - learning_rate * dm
        c = c - learning_rate * dc
        
        # Log history (sample effectively to avoid huge lists)
        if epochs <= 200 or i % (epochs // 50) == 0 or i == epochs - 1:
            loss = loss_function(y, y_pred)
            history['m'].append(m)
            history['c'].append(c)
            history['loss'].append(loss)
            history['epoch'].append(i)

    return m, c, history

# --- 2. Interactive Dashboard Class ---

class GradientDescentApp:
    def __init__(self):
        self.output = widgets.Output()
        
        # Default Sample Data (from your requirement)
        self.x_default = np.array([1, 2, 3, 4, 5], dtype=float)
        self.y_default = np.array([5, 7, 9, 11, 13], dtype=float)
        
        # Current Data
        self.x_data = self.x_default.copy()
        self.y_data = self.y_default.copy()
        
        # --- Controls ---
        self.lr_slider = widgets.FloatSlider(
            value=0.01, min=0.001, max=0.1, step=0.001,
            description='Learning Rate:',
            continuous_update=False,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='60%')
        )
        
        self.epochs_slider = widgets.IntSlider(
            value=1000, min=10, max=5000, step=10,
            description='Epochs:',
            continuous_update=False,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='60%')
        )
        
        self.reset_btn = widgets.Button(
            description="Reset Data", 
            button_style='info', 
            icon='refresh',
            layout=widgets.Layout(width='120px')
        )
        
        self.noise_btn = widgets.Button(
            description="Add Noise", 
            button_style='warning', 
            icon='random',
            tooltip="Add random noise to data to make optimization harder",
            layout=widgets.Layout(width='120px')
        )
        
        # Bind events
        self.lr_slider.observe(self.update_view, names='value')
        self.epochs_slider.observe(self.update_view, names='value')
        self.reset_btn.on_click(self.reset_data)
        self.noise_btn.on_click(self.add_noise)
        
        # Initial run
        self.update_view(None)

    def reset_data(self, _):
        """Resets to the original sample data [1,2,3,4,5], [5,7,9,11,13]"""
        self.x_data = self.x_default.copy()
        self.y_data = self.y_default.copy()
        self.update_view(None)

    def add_noise(self, _):
        """Adds random noise to the Y values to demonstrate robustness"""
        noise = np.random.normal(0, 1.0, len(self.x_data))
        self.y_data = self.y_default + noise
        self.update_view(None)

    def update_view(self, _):
        """Runs the optimization and updates the dashboard"""
        with self.output:
            self.output.clear_output(wait=True)
            
            # 1. Get Parameters
            lr = self.lr_slider.value
            epochs = self.epochs_slider.value
            
            # 2. Run Gradient Descent
            m_opt, c_opt, history = gradient_descent(self.x_data, self.y_data, lr, epochs)
            
            # 3. Check for Divergence (NaN or Infinity)
            if not np.isfinite(m_opt) or not np.isfinite(c_opt):
                display(HTML(f"""
                <div style="padding: 15px; background: #fee2e2; border-left: 5px solid #ef4444; border-radius: 5px;">
                    <h3 style="margin:0; color: #b91c1c;">⚠️ Optimization Diverged!</h3>
                    <p>The <b>Learning Rate ({lr})</b> is too high for this data/epoch combination, causing the gradients to explode.</p>
                    <p>Try reducing the Learning Rate (e.g., set it to 0.01).</p>
                </div>
                """))
                return

            # 4. Metrics Display
            final_loss = history['loss'][-1]
            timestamp = datetime.datetime.now().strftime("%H:%M:%S")
            
            display(HTML(f"""
            <div style="display: flex; gap: 20px; margin-bottom: 20px; align-items: center;">
                <div style="background: #f0f9ff; padding: 10px; border-radius: 8px; flex: 1; text-align: center; border: 1px solid #bae6fd;">
                    <small>Optimized Slope (m)</small><br>
                    <b style="font-size: 1.2em; color: #0284c7;">{m_opt:.4f}</b>
                </div>
                <div style="background: #f0f9ff; padding: 10px; border-radius: 8px; flex: 1; text-align: center; border: 1px solid #bae6fd;">
                    <small>Optimized Intercept (c)</small><br>
                    <b style="font-size: 1.2em; color: #0284c7;">{c_opt:.4f}</b>
                </div>
                <div style="background: #f0fdf4; padding: 10px; border-radius: 8px; flex: 1; text-align: center; border: 1px solid #bbf7d0;">
                    <small>Final Loss (MSE)</small><br>
                    <b style="font-size: 1.2em; color: #16a34a;">{final_loss:.5f}</b>
                </div>
            </div>
            <div style="text-align: right; color: #888; font-size: 0.8em;">Last Update: {timestamp}</div>
            """))
            
            # 5. Visualizations
            fig = make_subplots(
                rows=1, cols=2, 
                subplot_titles=("Regression Fit (Data vs Model)", "Loss Function Convergence")
            )
            
            # Plot A: Regression Line
            # Data points
            fig.add_trace(go.Scatter(
                x=self.x_data, y=self.y_data, 
                mode='markers', name='Data Points',
                marker=dict(size=12, color='#6366f1')
            ), row=1, col=1)
            
            # Best fit line
            x_range = np.linspace(min(self.x_data)-0.5, max(self.x_data)+0.5, 10)
            y_range = model(x_range, m_opt, c_opt)
            fig.add_trace(go.Scatter(
                x=x_range, y=y_range, 
                mode='lines', name=f'Fit: y={m_opt:.2f}x+{c_opt:.2f}',
                line=dict(color='#ef4444', width=3)
            ), row=1, col=1)
            
            # Plot B: Loss Curve
            fig.add_trace(go.Scatter(
                x=history['epoch'], y=history['loss'],
                mode='lines', name='Loss',
                line=dict(color='#22c55e', width=2)
            ), row=1, col=2)
            
            fig.update_layout(height=400, margin=dict(l=20, r=20, t=40, b=20), showlegend=True)
            fig.update_xaxes(title_text="X", row=1, col=1)
            fig.update_yaxes(title_text="Y", row=1, col=1)
            fig.update_xaxes(title_text="Epochs", row=1, col=2)
            fig.update_yaxes(title_text="MSE Loss", row=1, col=2)
            
            display(fig)

    def render(self):
        """Displays the main UI"""
        display(HTML("""
        <div style="background: linear-gradient(90deg, #4f46e5, #7c3aed); padding: 15px; border-radius: 8px; color: white; margin-bottom: 20px;">
            <h2 style="margin:0;">📉 Gradient Descent with NumPy</h2>
            <p style="margin:5px 0 0 0; opacity: 0.9;">Visualizing how the algorithm learns parameters m and c</p>
        </div>
        """))
        
        controls = widgets.VBox([
            widgets.HBox([self.lr_slider, self.epochs_slider]),
            widgets.HBox([self.reset_btn, self.noise_btn])
        ])
        
        display(widgets.VBox([controls, self.output]))

# --- Run Application ---
app = GradientDescentApp()
app.render()